In [1]:
%%HTML
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

In [3]:
import requests
from bs4 import BeautifulSoup


def url_to_soup(url):
    req = requests.get(url)
    return BeautifulSoup(req.content, 'html.parser')

In [4]:
bs = url_to_soup("https://www.nankankeiba.com/uma_info/2017103111.do")

In [5]:
bs

<?xml version="1.0" encoding="Shift_JIS"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="ja" http-equiv="content-language"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="telephone=no" name="format-detection"/>
<title>競走馬詳細データ | 南関東4競馬場 ｜nankankeiba.com</title>
<meta content="南関東4競馬,浦和競馬,船橋競馬,大井競馬,川崎競馬,リアルタイム情報,データ検索,開催日程,重賞競走,出馬表,オッズ,成績,決定番組,出走登録馬一覧,能力試験結果,持ち時計分析,種牡馬分析,交通アクセス" name="keywords">
<meta content="南関東4競馬の開催日程、重賞競走、出馬表、オッズ、成績、決定番組、出走登録馬一覧、能力試験結果、種牡馬分析、持ち時計の分析などの馬券検討情報" name="discription"/>
<link href="/shared/css/nankan_min.css?v=20200326_3" media="screen,projection" rel="stylesheet" type="text/css"/>
<link href="/shared/css/prt_nankan_min.css?v=2020

In [20]:
import re
import numpy as np
import pandas as pd
from IPython.core.display import display
p = re.compile(r'\<.*"\>|\<\/[a-z]+\>|\<[a-z]+\>|\<|\>|br') 
tag_to_text = lambda x: p.sub("", x).split('\n') 

split_tr = lambda x: str(x).split('</tr>')

def get_previous_race_row(soup):
    race_table = soup.select("table.tb01")[2]
    return [tag_to_text(x)  for x in split_tr(race_table)]

def horse_data(url, race_date):
    soup = url_to_soup(url)

    # 過去のレースデータ
    pre_race_data = get_previous_race_row(soup)
    df = pd.DataFrame(pre_race_data)[1:][[2,3,10,11,13,14,15,19,23]].dropna().rename(columns={
        2:'date', 3:'place', 10:'len', 11:'wether', 13:'popularity', 14:'rank', 15:'time',19:'weight',23:'money'})
    return df

In [22]:
pre_race_data = get_previous_race_row(bs)

In [23]:
df = pd.DataFrame(pre_race_data)

In [31]:
pd.set_option('display.max_columns',100)
display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,,,年月日,競馬場,R,レース名,距離,天候/馬場,馬番,人気,着順,タイム,差/事故,上3F,コーナー/通過順,体重,騎手,負担/重量,調教師,獲得賞金/（円）,,None,None,None,None
1,,,20/9/9,大井,1,,,３歳１８０万,,,1600,晴/稍重,5,1,4/12,1:45.0,0.9,41.8,3-2-2,493,的場文,56.0,渡部則,"240,000",
2,,,20/8/17,大井,1,,,ファンタスティックリレースタート賞 ３歳,,,1600,曇/良,11,1,2/14,1:44.2,0.1,40.4,7-5-5,493,岡部誠,56.0,渡部則,"640,000",
3,,,20/7/8,名古屋,2,,,名古屋モーニングフィーバー４ ３歳(四),,,1400,曇/不良,6,3,2/12,1:31.9,0.9,40.0,2-2-3,493,宮下瞳,54.0,安部幸,"102,000",
4,,,20/6/24,名古屋,2,,,３歳５組 ３歳(五),,,1400,曇/良,7,2,1/9,1:32.3,0.7,40.5,4-4-1,498,渡邊竜,56.0,安部幸,"360,000",
5,,,20/6/11,名古屋,4,,,３歳５組 ３歳(五),,,1400,雨/稍重,9,4,3/10,1:32.2,1.0,40.1,3-3-3,497,岡部誠,56.0,安部幸,"65,000",
6,,,20/5/28,名古屋,1,,,３歳８組 ３歳(八),,,1400,晴/良,1,2,1/9,1:31.3,0.2,39.1,4-3-2,493,宮下瞳,54.0,安部幸,"360,000",
7,,,20/5/15,名古屋,2,,,３歳９組 ３歳(九),,,1400,晴/良,4,1,2/9,1:33.0,1.1,40.7,5-5-5,497,宮下瞳,54.0,安部幸,"115,000",
8,,,20/4/12,中山,2,,\r,３歳\r,,,1200,曇/稍重,12,8,10/16,1:14.4,1.7,38.2,14-13,486,内田博,56.0,清水英,0,
9,,,20/3/1,中山,2,,\r,３歳\r,,,1200,晴/良,16,8,7/16,1:14.2,1.3,38.9,10-11,482,江田照,56.0,清水英,0,


In [32]:
display(df[1:])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
1,,,20/9/9,大井,1,,,３歳１８０万,,,1600,晴/稍重,5,1,4/12,1:45.0,0.9,41.8,3-2-2,493,的場文,56.0,渡部則,"240,000",
2,,,20/8/17,大井,1,,,ファンタスティックリレースタート賞 ３歳,,,1600,曇/良,11,1,2/14,1:44.2,0.1,40.4,7-5-5,493,岡部誠,56.0,渡部則,"640,000",
3,,,20/7/8,名古屋,2,,,名古屋モーニングフィーバー４ ３歳(四),,,1400,曇/不良,6,3,2/12,1:31.9,0.9,40.0,2-2-3,493,宮下瞳,54.0,安部幸,"102,000",
4,,,20/6/24,名古屋,2,,,３歳５組 ３歳(五),,,1400,曇/良,7,2,1/9,1:32.3,0.7,40.5,4-4-1,498,渡邊竜,56.0,安部幸,"360,000",
5,,,20/6/11,名古屋,4,,,３歳５組 ３歳(五),,,1400,雨/稍重,9,4,3/10,1:32.2,1.0,40.1,3-3-3,497,岡部誠,56.0,安部幸,"65,000",
6,,,20/5/28,名古屋,1,,,３歳８組 ３歳(八),,,1400,晴/良,1,2,1/9,1:31.3,0.2,39.1,4-3-2,493,宮下瞳,54.0,安部幸,"360,000",
7,,,20/5/15,名古屋,2,,,３歳９組 ３歳(九),,,1400,晴/良,4,1,2/9,1:33.0,1.1,40.7,5-5-5,497,宮下瞳,54.0,安部幸,"115,000",
8,,,20/4/12,中山,2,,\r,３歳\r,,,1200,曇/稍重,12,8,10/16,1:14.4,1.7,38.2,14-13,486,内田博,56.0,清水英,0,
9,,,20/3/1,中山,2,,\r,３歳\r,,,1200,晴/良,16,8,7/16,1:14.2,1.3,38.9,10-11,482,江田照,56.0,清水英,0,
10,,,20/1/25,中山,1,,\r,３歳\r,,,1200,曇/稍重,4,8,7/16,1:13.7,1.4,38.8,2-3,482,津村明,56.0,清水英,0,


In [39]:
 df = pd.DataFrame(pre_race_data)[1:][[2,3,10,11,13,14,15,19,23]].dropna().rename(columns={
        2:'date', 3:'place', 10:'len', 11:'wether', 13:'popularity', 14:'rank', 15:'time',19:'weight',23:'money'})

In [40]:
display(df
    )

,date,place,len,wether,popularity,rank,time,weight,money
1,20/9/9,大井,1600,晴/稍重,1,4/12,1:45.0,493,"240,000"
2,20/8/17,大井,1600,曇/良,1,2/14,1:44.2,493,"640,000"
3,20/7/8,名古屋,1400,曇/不良,3,2/12,1:31.9,493,"102,000"
4,20/6/24,名古屋,1400,曇/良,2,1/9,1:32.3,498,"360,000"
5,20/6/11,名古屋,1400,雨/稍重,4,3/10,1:32.2,497,"65,000"
6,20/5/28,名古屋,1400,晴/良,2,1/9,1:31.3,493,"360,000"
7,20/5/15,名古屋,1400,晴/良,1,2/9,1:33.0,497,"115,000"
8,20/4/12,中山,1200,曇/稍重,8,10/16,1:14.4,486,0
9,20/3/1,中山,1200,晴/良,8,7/16,1:14.2,482,0
10,20/1/25,中山,1200,曇/稍重,8,7/16,1:13.7,482,0


In [41]:
def result_data(url):
    soup = url_to_soup(url)

    # 土の状態
    condition = soup.find(id="race-data02").get_text().replace('\n','').split(';')[1].split('　')[2][0:1]

    # レースの長さ
    race_len = int(soup.find(id="race-data01-a").get_text().replace('\n','').split('　')[3].replace(',','')[1:5])

    # 1位
    hukusyo_list = []
    hukusyo_list.append(int(p.sub("", str(soup.find_all('tr', class_='bg-1chaku')[0]).split('</td>')[2]).replace('\n','') ))

    # レース日
    race_date_str = soup.find(id="race-data01-a").get_text().replace('\n','').split(';')[0].split('日')[0]
    race_date = dt.strptime(race_date_str, '%Y年%m月%d')
    return hukusyo_list, condition, race_len, race_date

In [42]:
result_data('https://www.nankankeiba.com/uma_shosai/2020092020100201.do')

IndexError: list index out of range

In [72]:
# 土の状態
soup = url_to_soup('https://www.nankankeiba.com/result/2020092020100201.do')
condition = soup.find(id="race-data02").get_text().split('\n')[3].split('　')[2]
condition

'不良・曇'

In [73]:
# レースの長さ
race_len = int(soup.find(id="race-data01-a").get_text().replace('\n','').split('　')[3].replace(',','')[1:5])
race_len

1600

In [75]:
# 1位
hukusyo_list = []
hukusyo_list.append(int(p.sub("", str(soup.find_all('tr', class_='bg-1chaku')[0]).split('</td>')[2]).replace('\n','') ))
hukusyo_list

[14]

In [78]:
hukusyo_list.append(p.sub("", str(soup.find_all('tr', class_='bg-1chaku')[0]).split('</td>')[3]).replace('\n','') )
hukusyo_list

[14, 'ナックライオン']

In [90]:
# 一位の馬番
tansyo_dict =  {}
name = p.sub("", str(soup.find_all('tr', class_='bg-1chaku')[0]).split('</td>')[3]).replace('\n','')
horse_number = int(p.sub("", str(soup.find_all('tr', class_='bg-1chaku')[0]).split('</td>')[2]).replace('\n','') )
tansyo_dict[name] = horse_number

In [91]:
tansyo_dict

{'ナックライオン': 14}

In [94]:
# レース日
from datetime import datetime
race_date_str = soup.find(id="race-data01-a").get_text().replace('\n','').split(';')[0].split('日')[0]
race_date = datetime.strptime(race_date_str, '%Y年%m月%d')
race_date

In [112]:
def add_soil_columns(row):
        row['soil_heavy'] = 1 if row['wether'][-2:] =='/重'  else 0
        row['soil_s_heavy'] = 1 if row['wether'][-2:] =='稍重'  else 0
        row['soil_good'] = 1 if row['wether'][-2:] =='/良'  else 0
        row['soil_bad'] = 1 if row['wether'][-2:] =='不良'  else 0
        return row

def add_race_data(df):
    df_ =pd.DataFrame()
    for idx, row in df.iterrows():
        if row['popularity'] == '':
            continue

        # 馬場状態
        row = add_soil_columns(row)

        row['money']=int(row['money'].replace(',','')) 
        row['horse_cnt'] = int(row['rank'].split('/')[1])
        row['result_rank'] = int(row['rank'].split('/')[0])
        row['len'] = int(row['len'][0:4])
        row['popularity'] = int(row['popularity'])
        row['weight'] = int(row['weight'])

        # 　競馬場の一致
        row['same_place'] = 1 if row['place'].startswith('大井')  else 0

        # タイム(秒)
        try:
            time = datetime.strptime(row['time'], '%M:%S.%f')
            row['sec'] = time.minute*60 + time.second + time.microsecond/1000000 
        except ValueError:
            time = datetime.datetime.strptime(row['time'], '%S.%f')
            row['sec'] = time.second + time.microsecond/1000000

        row['sec'] = int(row['sec']) 

        df_ = df_.append(row, ignore_index=True)
    df_ = df_.drop(columns=['place', 'rank','time','wether'])
    return df_

In [113]:
add_race_data(df)

,date,horse_cnt,len,money,popularity,result_rank,same_place,sec,soil_bad,soil_good,soil_heavy,soil_s_heavy,weight
0,20/9/9,12.0,1600.0,240000.0,1.0,4.0,1.0,105.0,0.0,0.0,0.0,1.0,493.0
1,20/8/17,14.0,1600.0,640000.0,1.0,2.0,1.0,104.0,0.0,1.0,0.0,0.0,493.0
2,20/7/8,12.0,1400.0,102000.0,3.0,2.0,0.0,91.0,1.0,0.0,0.0,0.0,493.0
3,20/6/24,9.0,1400.0,360000.0,2.0,1.0,0.0,92.0,0.0,1.0,0.0,0.0,498.0
4,20/6/11,10.0,1400.0,65000.0,4.0,3.0,0.0,92.0,0.0,0.0,0.0,1.0,497.0
5,20/5/28,9.0,1400.0,360000.0,2.0,1.0,0.0,91.0,0.0,1.0,0.0,0.0,493.0
6,20/5/15,9.0,1400.0,115000.0,1.0,2.0,0.0,93.0,0.0,1.0,0.0,0.0,497.0
7,20/4/12,16.0,1200.0,0.0,8.0,10.0,0.0,74.0,0.0,0.0,0.0,1.0,486.0
8,20/3/1,16.0,1200.0,0.0,8.0,7.0,0.0,74.0,0.0,1.0,0.0,0.0,482.0
9,20/1/25,16.0,1200.0,0.0,8.0,7.0,0.0,73.0,0.0,0.0,0.0,1.0,482.0
